En el presente trabajo se analizaron 3 algoritmos distintos con el objetivo de calcular los pesos de la matriz W. <br>

$WX = y$ <br>

$ W $ = matriz de pesos <br>
$ X $ = matriz de embeddings <br>
$ Y $ = matriz de resultados <br>

Se despejó la matriz $W$ calculando la Pseudo-Inversa de la matriz de embeddings $(X^{+})$, pero utilizando diferentes descomposiciones matriciales.

$W = YX^{+}$ <br>

#### Importamos librerías

In [53]:
import importlib
import alc
importlib.reload(alc)
import alcModulosNUMPY
importlib.reload(alcModulosNUMPY)

from alc import cargarDataset
from alcModulos import multiplicacion_de_matrices_sin_numpy, traspuesta

#### Lectura de Datos y separación de casos Train y Validation

In [2]:
Xt, Yt, Xv, Yv = cargarDataset("template-alumnos/cats_and_dogs")

---

A continuación, antes de presentar el punto 6, se calcula W mediante los 3 algortimos elaborados.

#### **Algoritmo 1**

In [ ]:
from alcModulos import calculaCholesky

A = multiplicacion_de_matrices_sin_numpy(Xt, traspuesta(Xt))
L = calculaCholesky(A)

In [ ]:
from alc import pinVEcuacionesNormales

# Cálculo de pseudoinversa de X y posteriormente W. (Devuelve solo W)
W_cholesky = pinVEcuacionesNormales(Xt, L, Yt)

#### **Algoritmo 2**

In [10]:
#Cálculo de la SVD de X con módulo:
from alcModulosNUMPY import svd_reducida

U, S, Vt = svd_reducida(Xt)

In [ ]:
from alc import pinvSVD

# Cálculo de pseudoinversa de X y posteriormente W. (Devuelve solo W)
W_SVD = pinvSVD(U, S, Vt, Yt)

---

### 6. Evaluación y Benchmarking
a. Para cada método de resolución de la W de los items anteriores, <br>
generar una matriz de confusión evaluando a partir de los pares de embeddings de validación o testing (Xv, Yv).

In [6]:
def accuracy(tp,tn,fp,fn):
    acertados = tp + tn
    total = tp + tn + fp + fn

    return ((acertados/total)*100)

In [7]:
def matriz_confusion(Y_pred):
    tp = 0  # Predice gato y era gato
    fp = 0  
    tn = 0  # Predice perro y era perro
    fn = 0  

    n = Y_pred.shape[1]
    for i in range(n):
        # aproximamois al que más se acerque ya que nunca va a dar exactamente valores pert a {0,1}
        # 0 = gato, 1 = perro
        if Y_pred[0, i] >= Y_pred[1, i]:
            pred = 0
        else:
            pred = 1

        if Yv[0, i] == 1:
            real = 0
        else:
            real = 1

        if pred == 0 and real == 0:
            tp += 1
        elif pred == 0 and real == 1:
            fp += 1
        elif pred == 1 and real == 1:
            tn += 1
        elif pred == 1 and real == 0:
            fn += 1

    return tp, tn, fp, fn


#### Algoritmo 1 - Ecuaciones Normales

In [8]:
#Calculamos predicciones de Y
Y_pred_Cholesky = W_cholesky @ Xv

#Calculamos la matriz de confusión
tp, tn, fp, fn = matriz_confusion(Y_pred_Cholesky)

matrizarriba = [[tp, fn]]
matrizabajo = [[fp, tn]]

print("Matriz de confusión Algoritmo 1 =")
print(matrizarriba)
print(matrizabajo)

#Calculamos accuracy
accuracyCholesky = accuracy(tp,tn,fp,fn)
print("Accuracy Algoritmo 1 =",accuracyCholesky,"%")

Matriz de confusión Algoritmo 1 =
[[334, 166]]
[[150, 350]]
Accuracy Algoritmo 1 = 68.4 %


#### Algoritmo 2 - SVD

In [54]:
#Calculamos predicciones de Y
Y_pred_SVD = W_SVD @ Xv

#Calculamos la matriz de confusión
tp, tn, fp, fn = matriz_confusion(Y_pred_SVD)

matrizarriba = [[tp, fn]]
matrizabajo = [[fp, tn]]

print("Matriz de confusión Algoritmo 2 =")
print(matrizarriba)
print(matrizabajo)

#Calculamos accuracy
accuracySVD = accuracy(tp,tn,fp,fn)
print("Accuracy Algoritmo 2 =",accuracySVD,"%")

Matriz de confusión Algoritmo 2 =
[[333, 167]]
[[152, 348]]
Accuracy Algoritmo 2 = 68.10000000000001 %


#### Algoritmo 3 - Descomposición QR

---

b. Presentar una tabla comparativa de los resultados de cada metodología donde en las columnas se debe mostrar la performance en clasificación.

| | Algoritmo 1 | Algoritmo 2 | Algortimo 3
|------|------|------|------|
| tiempo total (m)  | 118 | 41 | - |
| tiempo para  W (m)  | 23 | 0 | - |
| accuracy (%) | 68,4 | 68,1 | - |

#### Algortimo 1
El algoritmo 1 corrió en un tiempo de total de $\simeq$ 118 minutos.
- 94 minutos para calcular Xt @ Xt.T = A y calculaCholesky(A).
- 23 minutos para calcular W con pinVEcuacionesNormales().

Observaciones: <br>
- Se utilizó la traspuesta y multiplicación matricial de Numpy "@" y ".T" para obtener estos tiempos, tanto como para calcular "A" como dentro del algoritmo 1 "pinVEcuacionesNormales". <br>
- Dentro de la función calculaCholesky(A) se utilizó la implementación del grupo de la multiplicación matricial "multiplicacionMatricialConNumpy" y se cree que es por esto que tardó 94 minutos.


#### Algortimo 2
El algoritmo 2 corrió en un tiempo de total de $\simeq$ 41 minutos.
- 41 minutos para calcular la SVD reducida de Xt.
- 2.3 segundos para calcular W con pinvSVD().

Observaciones: <br>
- 


### 7. Síntesis Final